In [1]:
import re
import sys

In [2]:
def find_between( s, first, last ):
    try:
        start = s.index( first ) + len( first )
        end = s.index( last, start )
        return s[start:end]
    except ValueError:
        return ""
    
def comment_remover(text): #from: https://stackoverflow.com/questions/241327/python-snippet-to-remove-c-and-c-comments
    def replacer(match):
        s = match.group(0)
        if s.startswith('/'):
            return " " # note: a space and not an empty string
        else:
            return s
    pattern = re.compile(
        r'//.*?$|/\*.*?\*/|\'(?:\\.|[^\\\'])*\'|"(?:\\.|[^\\"])*"',
        re.DOTALL | re.MULTILINE
    )
    return re.sub(pattern, replacer, text)

def REGISTERED_Gen(ModuleName, Print_To_File, Files_Location):
    with open(Files_Location+ModuleName+".v","r") as v_file:
        code = v_file.read()
    #     print(code)
        c_removed = comment_remover(code)
    #     print(c_removed)
        c_removed = c_removed.replace("\n"," ") #for 'module ' (with space)
    #     print(c_removed)
        start = 'module '
        end = ';'
        data = find_between(c_removed,start,end)
        data = data.replace("#"," ")
        data = data.replace(","," ")
    #     print()
    #     print(data)

    splited = re.split("(parameter )|(input )|(output )|(=)|(\[.*?\])|(\()|(\))",  data)

    # print(splited)
    # print()

    valid_paramters = ["parameter ","input ","output "]
    PIO_list = []
    strings_list = []
    Start=False
    Count = 0
    strings = []
    for s in splited:
        if s!=None:
            if s in valid_paramters:
                if Start:
                    strings_list.append(strings)
                    strings = []
                Start = True
                Count += 1
                PIO_list.append(s[:-1])
            else:
                if Start:
                    for string in re.split(" ",s):
                        strings.append(string)
    strings_list.append(strings)

    # print(PIO_list)
    # print()
    # print(strings_list)
    # print()

    # for i in range(len(PIO_list)):
    #     print(PIO_list[i])
    #     print([x for x in strings_list[i] if x!="" and x!="(" and x!=")"])
    # print()

    hasClk = False
    signed_list = []
    size_list = []
    PIO_count_list = []
    PIO_names_list = []
    parameter_initial_list = []
    parameter_count = 0;
    input_count = 0;
    output_count = 0;
    parameter_l_count = 0;
    input_l_count = 0;
    output_l_count = 0;
    for i in range(len(PIO_list)):
        if PIO_list[i] == "parameter":
            parameter_l_count += 1;
            initialValue = False
            names_list = []
            signKeyword = False
            sizeString = False
            PIOcount = 0
            parameter_initials = []
            for x in strings_list[i]:
                if x=="":
                    continue
                elif initialValue:
                    parameter_initials.append(x)
                    initialValue = False
                elif x=="(" or x==")":
                    continue
                elif x=="signed":
                    signKeyword = True
                    signed_list.append(1)
                elif x=="unsigned":
                    signKeyword = True
                    signed_list.append(2) 
                elif x=="=":
                    initialValue = True
                elif x[0]=="[":
                    sizeString = True
                    size_list.append(x)
                else:
                    PIOcount += 1
                    names_list.append(x)
            if not signKeyword:
                signed_list.append(0)                                
            if not sizeString:
                size_list.append("")
            PIO_count_list.append(PIOcount)
            parameter_count += PIOcount;
            PIO_names_list.append(names_list)
            parameter_initial_list.append(parameter_initials)
        elif PIO_list[i] == "output":
            output_l_count += 1;
            initialValue = False
            names_list = []
            signKeyword = False
            sizeString = False
            PIOcount = 0
            for x in strings_list[i]:
                if x=="":
                    continue
                elif initialValue:
                    initialValue = False
                elif x=="(" or x==")" or x=="reg":
                    continue
                elif x=="signed":
                    signKeyword = True
                    signed_list.append(1)
                elif x=="unsigned":
                    signKeyword = True
                    signed_list.append(2)
                elif x=="=":
                    initialValue = True
                elif x[0]=="[":
                    sizeString = True
                    size_list.append(x)
                else:
                    PIOcount += 1
                    names_list.append(x)
            if not signKeyword:
                signed_list.append(0)                                
            if not sizeString:
                size_list.append("")
            PIO_count_list.append(PIOcount)
            output_count += PIOcount;
            PIO_names_list.append(names_list)
        elif PIO_list[i] == "input":
            input_l_count += 1;
            names_list = []
            signKeyword = False
            sizeString = False
            PIOcount = 0
            for x in strings_list[i]:
                if x=="" or x=="(" or x==")":
                    continue
                elif x=="clk":
                    hasClk = True
                elif x=="signed":
                    signKeyword = True
                    signed_list.append(1)
                elif x=="unsigned":
                    signKeyword = True
                    signed_list.append(2)
                elif x[0]=="[":
                    sizeString = True
                    size_list.append(x)
                else:
                    PIOcount += 1
                    names_list.append(x)
            if not signKeyword:
                signed_list.append(0)
            if not sizeString:
                size_list.append("")
            PIO_count_list.append(PIOcount)
            input_count += PIOcount;
            PIO_names_list.append(names_list)

    # for i in range(len(PIO_list)):
    #     print("type: " + PIO_list[i])
    #     print("sign type: {}".format(signed_list[i]))
    #     print("size string: " + size_list[i])
    #     print("count: {}".format(PIO_count_list[i]))
    #     print("names: ", end="")
    #     print(PIO_names_list[i])
    #     #if PIO_list[i] == "parameter":
    #     if i < parameter_l_count:
    #         print("parameter initials: ", end="")
    #         print(parameter_initial_list[i])
    #     print()

    # print("has Clk: ", end="")
    # print(hasClk)
    # print()

    ## ModuleName = splited[0].strip()
    # print("module name: " + ModuleName)

    if Print_To_File<=0:
        of=sys.stdout
    else:
        of=open(Files_Location+ModuleName+"_REGISTERED.v", 'w+')

    print("`timescale 1ns / 1ps\n", file=of)
    print("module "+ModuleName+"_REGISTERED", file=of)
    print("#(", file=of)
    for i in range(parameter_l_count):
        print("parameter ", end='', file=of)
        if signed_list[i]==1:
            print("signed ", end='', file=of)
        elif signed_list[i]==2:
            print("unsigned ", end='', file=of)
        if size_list[i]!="":
            print(size_list[i]+" ", end='', file=of)
        for j in range(PIO_count_list[i]):
            print(PIO_names_list[i][j]+" = {}".format(parameter_initial_list[i][j]), end='', file=of)
            if i!=parameter_l_count-1 or j!=PIO_count_list[i]-1:
                print(", ", end='', file=of)
        print(file=of)
    print(")(", file=of)
    print("input clk,", file=of)
    io_l_count = input_l_count + output_l_count
    io_l_end = parameter_l_count+io_l_count
    for i in range(parameter_l_count, io_l_end):
        if PIO_list[i]=="input":
            is_input = True
            print("input ", end='', file=of)
        else:
            is_input = False
            print("output reg ", end='', file=of)
        if signed_list[i]==1:
            print("signed ", end='', file=of)
        elif signed_list[i]==2:
            print("unsigned ", end='', file=of)
        if size_list[i]!="":
            print(size_list[i]+" ", end='', file=of)
        #io_c = PIO_count_list[i]
        for j in range(PIO_count_list[i]):
            print(PIO_names_list[i][j], end='', file=of)
    #         if j==(PIO_count_list[i]-1) and i==(io_l_end-1):
    #             print(file=of)
    #         elif j!=(PIO_count_list[i]-1):
    #             print(", ", end='', file=of)            
    #         else:
    #             print(", ", file=of)
            if i!=io_l_end-1 or j!=PIO_count_list[i]-1:
                print(", ", end='', file=of)
        print(file=of)
    print(");", file=of)
    print(file=of)

    for i in range(parameter_l_count, io_l_end):
        if PIO_list[i]=="input":
            is_input = True
            print("reg ", end='', file=of)
        else:
            is_input = False
            print("wire ", end='', file=of)
        if signed_list[i]==1:
            print("signed ", end='', file=of)
        elif signed_list[i]==2:
            print("unsigned ", end='', file=of)
        if size_list[i]!="":
            print(size_list[i]+" ", end='', file=of)
        for j in range(PIO_count_list[i]):
            print(PIO_names_list[i][j], end='', file=of)
            if is_input:
                print("_R", end='', file=of)
            else:
                print("_W", end='', file=of)
            if j==(PIO_count_list[i]-1):
                print(";", file=of)
            else:
                print(",", end='', file=of)
    print(file=of)

    print("always @(posedge clk) begin", file=of)
    for i in range(parameter_l_count, io_l_end):
        if PIO_list[i]=="input":
            is_input = True
        else:
            is_input = False
        print("\t", end='', file=of)
        for j in range(PIO_count_list[i]):
            if is_input:
                print(PIO_names_list[i][j]+"_R <= "+PIO_names_list[i][j]+"", end='', file=of)
            else:
                print(PIO_names_list[i][j]+" <= "+PIO_names_list[i][j]+"_W", end='', file=of)
            print("; ", end='', file=of)
        print(file=of)
    print("end", file=of)
    print(file=of)

    print(ModuleName, file=of)
    print("#(", file=of)
    for i in range(parameter_l_count):
        for j in range(PIO_count_list[i]):
            print(PIO_names_list[i][j], end='', file=of)
            if i!=parameter_l_count-1 or j!=PIO_count_list[i]-1:
                print(", ", end='', file=of)
        print(file=of)

    print(")Rinst(", file=of)
    if hasClk:
        print("clk,", file=of)    
    for i in range(parameter_l_count, io_l_end):
        if PIO_list[i]=="input":
            is_input = True
        else:
            is_input = False

        for j in range(PIO_count_list[i]):
            print(PIO_names_list[i][j], end='', file=of)
            if is_input:
                print("_R", end='', file=of)
            else:
                print("_W", end='', file=of)
            if i!=io_l_end-1 or j!=PIO_count_list[i]-1:
                print(", ", end='', file=of)
        print(file=of)
    print(");", file=of)
    print(file=of)

    print("endmodule", file=of)

    if Print_To_File>0:
        of.close()